In [1]:
import collections
import matplotlib.mlab as mlab
import numpy as np
import scipy as sp
from scipy import optimize
import os
import copy
import sys
import mosek
import pickle
from sys import argv
from datetime import datetime

In [2]:
class species(object):
    def __init__(self, Nres, strat_dist, lin, s, strat_p=0.1):
        if strat_dist == 'uniform':
            self.strategy = [np.random.uniform() for i in range(Nres)]
        elif strat_dist == 'binomial':
            self.strategy = np.random.choice([0, 1], size=(Nres), p=[1-strat_p, strat_p])
        else:
            print 'Incorrectly specified distribution: options include "uniform" and "binomial".'
        if np.sum(self.strategy)!= 0:
            self.strat_norm = self.strategy/np.float64(np.sum(self.strategy))
        else:
            self.strat_norm = np.float64(self.strategy)
        self.fitness = np.exp(np.random.normal(0, s))
    
    def perturb_fitness(self, s):
        self.fitness = self.fitness*np.exp(s)
        return
    
    def perturb_strat_dir(self, idx):
        self.strategy[idx] = 1-self.strategy[idx]
        if np.sum(self.strategy)!= 0:
            self.strat_norm = self.strategy/np.float64(np.sum(self.strategy))
        else:
            self.strat_norm = np.float64(self.strategy)
        return
    
    def generalist(self, Nres):
        self.strategy = np.ones(Nres)
        self.strat_norm = self.strategy/np.float64(np.sum(self.strategy))
        return

    def specialist(self, Nres, idx):
        self.strategy = np.zeros(Nres)
        self.strategy[idx] = 1
        self.strat_norm = self.strategy
        return
    
    def specify_strat(self, strat):
        self.strategy = strat
        self.strat_norm = self.strategy/np.float64(np.sum(self.strategy))
        return

In [3]:
def row_sparsify(A):
    asub = []
    aspec = []
    for i in range(len(A)):
        asub.append([])
        aspec.append([])
        for j in range(len(A[i])):
            if A[i][j] > 0:
                asub[i].append(j)
                aspec[i].append(A[i][j])
    return asub, aspec

def run_opt(vspecies, vres):
    Nspec = len(vspecies)
    Nres = len(vres)
    
    with mosek.Env() as env:
        with env.Task(0,0) as task:
            
            task.putdouparam(mosek.dparam.intpnt_nl_tol_dfeas,0.0)
            task.putdouparam(mosek.dparam.intpnt_nl_tol_rel_gap,1e-14)
            task.putdouparam(mosek.dparam.intpnt_nl_tol_pfeas,0.0)
            
            numvar = Nres
            numcon = Nspec
            
            inf = 0.0
            
            bkx = [ mosek.boundkey.lo ] * numvar
            blx = [ 0.0 ] * numvar
            bux = [ inf ] * numvar
            
            bkc = [ mosek.boundkey.up ] * numcon
            buc = [1.0/vspecies[i].fitness for i in range(Nspec)]
            
            task.appendvars(numvar)
            task.appendcons(numcon)
            
            task.putvarboundslice(0, numvar, bkx, blx, bux)
            
            A = [vspecies[i].strat_norm for i in range(Nspec)]
            asub, aval = row_sparsify(A)
            
            for i in range(numcon):
                task.putbound(mosek.accmode.con,i,bkc[i],-inf,buc[i])
                task.putarow(i, asub[i], aval[i]);
        
            opro  = [mosek.scopr.log for i in range(Nres)]
            oprjo = range(Nres)
            oprfo = -vres
            oprgo = [1.0 for i in range(Nres)]
            oprho = [0.0 for i in range(Nres)]
            
            task.putSCeval(opro, oprjo, oprfo, oprgo, oprho)
            
            task.optimize()
            
            res = [ 0.0 ] * numvar
            task.getsolutionslice(mosek.soltype.itr,
                                  mosek.solitem.xx,
                                  0, numvar,
                                  res)
                                  
            delta = np.array(A).dot(np.array(res))-np.array(buc)

            return res, delta

In [5]:
# Run random environment with random initial species distribution
def run_rand_env(Nres, alpha, eps, s):
    Nspec = int(alpha*Nres)
    
    #Normalize influx for rescaled equations
    vres = np.ones(Nres)+np.random.normal(0, eps, size=Nres)
    vres = vres/sum(vres)
    vspecies = [species(Nres, 'binomial', i, s) for i in range(Nspec)]
    h, delta = run_opt(vspecies, vres)
    
    # Numerical precision is 1e-6, so for |delta|<1e-6,
    # set to zero.
    present = []
    absent = []
    
    for i in range(Nspec):
        if np.abs(delta[i])<1e-8:
            present.append(vspecies[i])
        else:
            absent.append(vspecies[i])
    return vres, h, present

# Run specified environment
def run_spec_env(vres, vspecies):
    Nspec = len(vspecies)
    
    #run optimization scheme
    if Nspec == 1:
        h = [vres[i]/vspecies[0].strat_norm[i] for i in range(len(vres))]
        h = h/vspecies[0].fitness
        delta = [0]
    else:
        h, delta = run_opt(vspecies, vres)
    
    #presence/absence vector
    present = []
    
    #For |delta|<1e-9, set to zero.
    for i in range(Nspec):
        if np.abs(delta[i])<1e-9:
            present.append(vspecies[i])
    return vres, h, present

# Start generalist
def start_generalist(Nres, eps, s):
    #Normalize influx for rescaled equations
    vres = np.ones(Nres)+np.random.normal(0, eps, size=Nres)
    vres = vres/sum(vres)
    
    vspecies = [species(Nres, 'binomial', 0, s)]
    vspecies[0].generalist(Nres)
    
    h = [vres[i]/vspecies[0].strat_norm[i] for i in range(len(vres))]
    h = h/vspecies[0].fitness
    
    return vres, h, vspecies

# Start specialist
def start_specialist(Nres, eps, s):
    #Normalize influx for rescaled equations
    vres = np.ones(Nres)+np.random.normal(0, eps, size=Nres)
    vres = vres/sum(vres)
    
    vspecies = [species(Nres, 'binomial', i, 0.1) for i in range(Nres)]
    for i in range(Nres):
        vspecies[i].specialist(Nres, i)
    
    h = [1/vspecies[i].fitness for i in range(len(vres))]
    
    return vres, h, vspecies

In [6]:
def calculate_frequencies(present, vres, h):
    a = np.array([present[i].fitness*present[i].strat_norm for i in range(len(present))])
    a = np.transpose(a)
    b = np.array([vres[i]/h[i] for i in range(len(vres))])
    m = sp.optimize.nnls(a, b)[0]
    return m

def calc_inv_fit(spec, h):
    return spec.fitness*np.dot(spec.strat_norm, h)-1

def pick_species(rates, spec_pool):
    norm = np.sum(rates)
    idxs = range(len(spec_pool))
    idx = np.random.choice(idxs, p=rates/norm)
    return spec_pool[idx], norm

def create_pool(vspecies, vres, h, freq, s, strat_prob, fit_prob):
    
    pool = []
    inv_rates = []
    for i in range(len(vspecies)):
        
        #fitness mutants
        temp = copy.deepcopy(vspecies[i])
        temp.perturb_fitness(s)
        pool.append(temp)
        inv_rates.append(calc_inv_fit(temp, h)*freq[i]*fit_prob)

        #strategy mutants
        for j in range(len(vres)):
            temp = copy.deepcopy(vspecies[i])
            temp.perturb_strat_dir(j)
            temp_rate = calc_inv_fit(temp,h)*freq[i]*strat_prob
            if temp_rate>0.0:
                pool.append(temp)
                inv_rates.append(temp_rate)

    return pool, inv_rates

def sswm_evol(vspecies, vres, h, s, strat_prob, fit_prob):
    #calculate frequencies
    freq = calculate_frequencies(vspecies, vres, h)
    #generate mutant pool and pick mutant
    pool, inv_rates = create_pool(vspecies, vres, h, freq, s, strat_prob, fit_prob)
    temp, time = pick_species(inv_rates, pool)
    vspecies.append(temp)
    return vspecies, freq, time

def runT(ic, Nres, alpha, T, s, eps, fit_rate):
    
    data = []

    #initialize environment
    if ic=="generalist":
        vres, h, present = start_generalist(Nres, eps, s)
    elif ic=="specialist":
        vres, h, present = start_specialist(Nres, eps, s)
    else:
        vres, h, present = run_rand_env(Nres, alpha, eps, s)
    
    strat_rate = 1.0
    fit_rate = float(fit_rate)
    
    #run timecourse
    for i in range(T):
        present, freq, time = sswm_evol(present, vres, h, s, strat_rate, fit_rate)
        vres, h, present = run_spec_env(vres, present)
        if i > T-10*Nres:
            data.append([vres, h, present, time])

    #return final population
    return data

def create_pool_rmax(vspecies, vres, h, freq, s, strat_prob, fit_prob, exchange_prob, Rmax):
    
    pool = []
    inv_rates = []
    for i in range(len(vspecies)):
        
        #fitness mutants
        temp = copy.deepcopy(vspecies[i])
        temp.perturb_fitness(s)
        pool.append(copy.deepcopy(temp))
        inv_rates.append(calc_inv_fit(temp, h)*freq[i]*fit_prob)

        #strategy mutants
        for j in range(len(vres)):
            temp = copy.deepcopy(vspecies[i])
            temp.perturb_strat_dir(j)
            temp_rate = calc_inv_fit(temp,h)*freq[i]*strat_prob
            if temp_rate>0.0 and sum(temp.strategy)<=Rmax:
                pool.append(temp)
                inv_rates.append(temp_rate)
            for k in range(len(vres)):
                if temp.strategy[j]-temp.strategy[k]==0 and k>j:
                    temp2 = copy.deepcopy(temp)
                    temp2.perturb_strat_dir(k)
                    temp_rate = calc_inv_fit(temp2,h)*freq[i]*exchange_prob
                    if temp_rate>0.0 and sum(temp2.strategy)<=Rmax:
                        pool.append(temp2)
                        inv_rates.append(temp_rate)

    return pool, inv_rates

def sswm_evol_rmax(vspecies, vres, h, s, strat_prob, fit_prob, exchange_prob, Rmax):
    #calculate frequencies
    freq = calculate_frequencies(vspecies, vres, h)
    #generate mutant pool and pick mutant
    pool, inv_rates = create_pool_rmax(vspecies, vres, h, freq, s, strat_prob, fit_prob, exchange_prob, Rmax)
    temp = pick_species(inv_rates, pool)
    vspecies.append(temp)
    return vspecies, freq

def runT_rmax(ic, Nres, alpha, T, s, eps, fit_rate, Rmax):
    
    data = []
    
    #initialize environment
    if ic=="specialist":
        vres, h, present = start_specialist(Nres, eps, s)
    else:
        vres, h, present = run_rand_env(Nres, alpha, eps, s, Rmax)
    
    strat_rate = 1.0
    exchange_rate = 0.01
    fit_rate = float(fit_rate)
    
    #run timecourse
    for i in range(T):
        present, freq = sswm_evol_rmax(present, vres, h, s, strat_rate, fit_rate, exchange_rate, Rmax)
        vres, h, present = run_spec_env(vres, present)
        if i > T-10*Nres:
            data.append([vres, h, present])
    
    #return final population
    return data

In [ ]:
#generate data for figure 4
Nres = [25, 50, 100]
alpha = 6
T = 100000
eps = 1e-3
s = 1e-7
ratio = [1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0, 10000000.0, 100000000.0, 1000000000.0]
ic = ['generalist', 'specialist', 'random']
data = []

for n in Nres:
    for r in ratio:
        for i in ic:
            data.append(run_T(ic, Nres, alpha, T, s, eps, r))

In [ ]:
#generate data for rmax simulations
Nres = [50]
Rmax = 5
alpha = 6
T = 100000
eps = 1e-3
s = 1e-7
ratio = [1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0, 10000000.0, 100000000.0, 1000000000.0]
ic = ['generalist', 'specialist', 'random']
data = []

for n in Nres:
    for r in ratio:
        for i in ic:
            data.append(run_T_rmax(ic, Nres, alpha, T, s, eps, r, Rmax))